In [ ]:
# 環境初始化 (大約三至五分鐘)
! wget -O init_env.sh https://www.dropbox.com/s/6bnwn8u2hz19s59/init_env.sh && \
bash init_env.sh

!which python
!/usr/local/bin/python -V

import os, sys

os.environ['SPARK_HOME'] = "/usr/local/spark"
# os.environ['PYSPARK_PYTHON'] = "/usr/bin/python"
os.environ['PYSPARK_PYTHON'] = "/usr/local/bin/python"
sys.path.append("/usr/local/spark/python/")
sys.path.append("/usr/local/spark/python/lib/pyspark.zip")
sys.path.append("/usr/local/spark/python/lib/py4j-0.10.4-src.zip")

In [2]:
################## spark declare #######################
from pyspark import SparkContext
from pyspark import SparkConf

#sc.stop()
sc = SparkContext()

In [ ]:
############ import ###########################
import json
import jieba
import jieba.posseg as pseg

import time

############## funcution ######################
def specific_find(x):
  temp_title = x.strip().split("\t", 1)[0]

  for n_ in find_option:
    if (temp_title == n_):
      return x
#################################################

##################### main #####################
tick = time.time()

jieba.set_dictionary('/content/drive/My Drive/ques/dictionary/dict.txt.big.txt')

with open('/content/drive/My Drive/Questions_with_Ans.json','r',encoding='UTF-8') as f:
 JsonList = json.load(f)

data_file = '/content/drive/My Drive/index_final2'


text_file = sc.textFile(data_file)

ans = []
ans_1 = []
find_option = []
find_ques = []
find_ques_1 = []
#################### word find candicate #####################
for k in range(200):
  temp = set()

  a = JsonList[k]['A']
  b = JsonList[k]['B']
  c = JsonList[k]['C']

  find_option.append(a)
  find_option.append(b)
  find_option.append(c)

  temp_relation = JsonList[k]['Question']

  relation_ori = pseg.cut(temp_relation)

  for a_, b_ in relation_ori:

    if (b_.find("x") >= 0 or b_.find("n") >= 0):

      find_option.append(a_)

tempset = set(find_option)
find_option = list(tempset)

#################### spark #####################################

index_freq = text_file.filter(specific_find).collect()

####################### answer #################################
print("[")

for m in range(200):
  a = JsonList[m]['A']
  b = JsonList[m]['B']
  c = JsonList[m]['C']

  temp_relation = []
  temp_relation_1 = JsonList[m]['Question']
    
  print("ques: ", temp_relation_1)
  print("A: ", a)
  print("B: ", b)
  print("C: ", c)

  relation_ori = pseg.cut(temp_relation_1)
  
  for a_, b_ in relation_ori:
    if (b_.find("n") >= 0 or b.find("x") >= 0):
      temp_relation.append(a_)

  temp = set(temp_relation)
  temp_relation = list(temp)

  a_list = []
  b_list = []
  c_list = []
  d_list = []

  a_set = set()
  b_set = set()
  c_set = set()
  d_set = set()

  a_freq = 0
  b_freq = 0
  c_freq = 0


  for m_ in index_freq:
    title, index = m_.split("\t")

  #--------------------- option ------------------------------
    if (title == a):

      index2 = index.split(" ")

      for n__ in index2:

        a_list.append(n__)
        a_freq += int(n__.split(":")[1])
        a_set.add(n__.split(":")[0])
      
      continue

    if (title == b):

      index2 = index.split(" ")

      for n__ in index2:

        b_list.append(n__)
        b_freq += int(n__.split(":")[1])
        b_set.add(n__.split(":")[0])

      continue

    if (title == c):
      
      index2 = index.split(" ")

      for n__ in index2:

        c_list.append(n__)
        c_freq += int(n__.split(":")[1])
        c_set.add(n__.split(":")[0])

      continue

  for d_ in temp_relation:
    for m_ in index_freq:
      title, index = m_.split("\t")

      if (title == d_):
        index3 = index.split(" ")
        if (len(d_set) > 0):

          tempset = set()
          for n__ in index3:

            d_list.append(n__)
            tempset.add(n__.split(":")[0])

          d_set = d_set & tempset
          break
        else:
          for n__ in index3:

            d_list.append(n__)
            d_set.add(n__.split(":")[0])
          break
  #-------------------------------------------------------

  a_set_temp = set()
  b_set_temp = set()
  c_set_temp = set()

  a_set_temp = list(a_set & d_set)
  b_set_temp = list(b_set & d_set)
  c_set_temp = list(c_set & d_set)

  a_count = 0
  b_count = 0
  c_count = 0

  if (a_set_temp == [] and b_set_temp == [] and c_set_temp == []):

    if (b_freq > a_freq):
      if (b_freq > c_freq):
        ans.append("B")
        print("MyAns: B")
      else:
        ans.append("C")
        print("MyAns: C")
    elif (c_freq > a_freq):
      ans.append("C")
      print("MyAns: C")
    else:
      ans.append("A")
      print("MyAns: A")
  else:
    # Handle wiki_index:wiki_count and calculate wiki_count
    for jj in a_set_temp:
      for jjj in a_list:
        index_a, count_a = jjj.split(":")
        if (index_a == jj):
          a_count += int(count_a)
          break
    for jj in b_set_temp:
      for jjj in b_list:
        index_b, count_b = jjj.split(":")
        if (index_b == jj):
          b_count += int(count_b)
          break
    for jj in c_set_temp:
      for jjj in c_list:
        index_c, count_c = jjj.split(":")
        if (index_c == jj):
          c_count += int(count_c)
          break

    if (b_count > a_count):
      if (b_count > c_count):
        ans.append("B")
        print("MyAns: B")

      else:
        ans.append("C")
        print("MyAns: C")

    elif (c_count > a_count):
      ans.append("C")
      print("MyAns: C")

    else:
      ans.append("A")
      print("MyAns: A")

  a_set = set()
  b_set = set()
  c_set = set()
  d_set = set()

tick1 = time.time()

In [7]:
RealAns = []
count = 0

for k in range(200):

  # a = JsonList[k]['A']
  # b = JsonList[k]['B']
  # c = JsonList[k]['C']

  RealAns.append(JsonList[k]['Answer'])

  if (ans[k] == JsonList[k]['Answer']):
    count += 1

############ ans2json ###################
ans1 = json.dumps(ans)

############ show ans ##################
print("Real ans:", RealAns)
print("Our ans: ", ans1)
print("correct: ", count)
print("correct rate: ", (count / 200))
print("Spending time: ", tick1 - tick, "s")

Real ans: ['B', 'C', 'B', 'A', 'C', 'A', 'A', 'C', 'B', 'B', 'A', 'C', 'C', 'B', 'A', 'B', 'A', 'C', 'B', 'B', 'C', 'A', 'C', 'A', 'A', 'C', 'B', 'C', 'A', 'B', 'C', 'A', 'C', 'C', 'A', 'B', 'A', 'C', 'B', 'A', 'B', 'C', 'A', 'B', 'A', 'C', 'A', 'B', 'A', 'C', 'B', 'B', 'B', 'C', 'C', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'B', 'C', 'C', 'A', 'B', 'C', 'B', 'C', 'B', 'B', 'B', 'A', 'B', 'C', 'A', 'A', 'B', 'A', 'C', 'B', 'B', 'C', 'A', 'C', 'B', 'C', 'A', 'A', 'B', 'A', 'A', 'C', 'B', 'C', 'C', 'A', 'B', 'B', 'A', 'A', 'C', 'B', 'B', 'A', 'C', 'B', 'B', 'B', 'A', 'A', 'A', 'C', 'B', 'A', 'B', 'B', 'A', 'C', 'C', 'A', 'B', 'C', 'B', 'A', 'A', 'B', 'B', 'C', 'C', 'B', 'A', 'B', 'C', '', 'A', 'C', 'A', 'A', 'B', 'C', 'B', 'A', 'A', 'C', 'A', 'A', 'B', 'A', 'B', 'A', 'C', 'B', 'C', 'A', 'A', 'B', 'A', 'C', 'A', 'A', 'B', 'A', 'C', 'A', 'C', 'C', 'A', 'B', 'A', 'C', 'B', 'C', 'A', 'B', 'A', 'C', 'B', 'A', 'C', 'A', 'B', 'A', 'B', 'B', 'B', 'C', 'C', 'A', 'A', 'B', 'B', 'B', 